In [ ]:
import pandas as pd
from datetime import datetime
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.utils.dataframe import dataframe_to_rows

# Lấy dữ liệu
quote = Quote(symbol="VNM", source="VCI")
price_data = quote.history(start='2020-10-03', end='2025-10-03', interval='1D')

finance_data = Finance(symbol="VNM", source="VCI")
business_data = finance_data.income_statement(period='year', lang='vi')

# Tạo thư mục
if not os.path.exists('data'):
    os.makedirs('data')

# Tên file
today = datetime.now().strftime("%d-%m-%y")
filename = f"data/VNM_tong_hop_{today}.xlsx"

# Gộp vào 1 file Excel
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    price_data.to_excel(writer, sheet_name='Giá_lịch_sử', index=False)
    business_data.to_excel(writer, sheet_name='Báo_cáo_KQKD', index=False)

# Format đẹp cho file Excel
wb = load_workbook(filename)

# Format sheet "Giá_lịch_sử"
if 'Giá_lịch_sử' in wb.sheetnames:
    ws_price = wb['Giá_lịch_sử']
    
    # Format header
    header_font = Font(bold=True, color="FFFFFF", size=11)
    header_fill = PatternFill(start_color="2F5597", end_color="2F5597", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    
    for cell in ws_price[1]:  # Dòng header
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    
    # Format cột ngày
    for cell in ws_price['A']:
        if cell.row > 1:
            cell.number_format = 'dd/mm/yyyy'
            cell.alignment = Alignment(horizontal="center")
    
    # Format cột số (giá, volume)
    for col in ['B', 'C', 'D', 'E', 'F']:  # open, high, low, close, volume
        for cell in ws_price[col]:
            if cell.row > 1:
                cell.number_format = '#,##0.00'
                cell.alignment = Alignment(horizontal="right")
    
    # Tự động điều chỉnh độ rộng cột
    for column in ws_price.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws_price.column_dimensions[column_letter].width = adjusted_width
    
    # Thêm border
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    for row in ws_price.iter_rows():
        for cell in row:
            cell.border = thin_border
    
    # Alternating row colors
    light_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    for row in range(2, ws_price.max_row + 1):
        if row % 2 == 0:
            for col in range(1, ws_price.max_column + 1):
                ws_price.cell(row=row, column=col).fill = light_fill

# Format sheet "Báo_cáo_KQKD"
if 'Báo_cáo_KQKD' in wb.sheetnames:
    ws_business = wb['Báo_cáo_KQKD']
    
    # Format header
    for cell in ws_business[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    
    # Format cột năm
    for cell in ws_business['B']:  # Cột năm
        if cell.row > 1:
            cell.alignment = Alignment(horizontal="center")
            cell.font = Font(bold=True)
    
    # Format cột phần trăm
    percent_columns = ['C', 'F']  # Tăng trưởng doanh thu, lợi nhuận
    for col in percent_columns:
        for cell in ws_business[col]:
            if cell.row > 1:
                cell.number_format = '0.00"%"'
                cell.alignment = Alignment(horizontal="right")
    
    # Format cột số tiền (tỷ VND)
    money_columns = ['D', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
    for col in money_columns:
        for cell in ws_business[col]:
            if cell.row > 1:
                cell.number_format = '#,##0.00" tỷ"'
                cell.alignment = Alignment(horizontal="right")
    
    # Tự động điều chỉnh độ rộng cột
    for column in ws_business.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 25)
        ws_business.column_dimensions[column_letter].width = adjusted_width
    
    # Thêm border
    for row in ws_business.iter_rows():
        for cell in row:
            cell.border = thin_border
    
    # Alternating row colors
    for row in range(2, ws_business.max_row + 1):
        if row % 2 == 0:
            for col in range(1, ws_business.max_column + 1):
                ws_business.cell(row=row, column=col).fill = light_fill
    
    # Conditional formatting cho phần trăm tăng trưởng
    green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
    red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    
    # Tăng trưởng doanh thu
    for row in range(2, ws_business.max_row + 1):
        cell = ws_business[f'C{row}']
        if cell.value and cell.value > 0:
            cell.fill = green_fill
        elif cell.value and cell.value < 0:
            cell.fill = red_fill
    
    # Tăng trưởng lợi nhuận
    for row in range(2, ws_business.max_row + 1):
        cell = ws_business[f'F{row}']
        if cell.value and cell.value > 0:
            cell.fill = green_fill
        elif cell.value and cell.value < 0:
            cell.fill = red_fill

# Lưu file
wb.save(filename)
print(f"Đã tạo file đẹp: {filename}")